In [ ]:
%pip install tensorflow

In [11]:
import os, glob, zipfile, json, tensorflow as tf
print("TF:", tf.__version__)

TF: 2.20.0


In [12]:
#fetch from Google Drive, extract, and locate `mrleyedataset`
%pip -q install gdown

import os, zipfile
from glob import glob
import gdown

FILE_ID = "1Zr_AtNJsZ1a4QNipHvAGddCc7XMUJUUX"   # <-- change if needed
OUT_ZIP = "/content/archive.zip"
DATA_ROOT = "/content/data"
TARGET_NAME = "mrleyedataset"
EXTRACT_DIR = os.path.join(DATA_ROOT, TARGET_NAME)

gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", OUT_ZIP, quiet=False)
os.makedirs(DATA_ROOT, exist_ok=True)

with zipfile.ZipFile(OUT_ZIP, "r") as zf:
    zf.extractall(DATA_ROOT)

if not os.path.isdir(EXTRACT_DIR):
    candidates = [p for p in glob(os.path.join(DATA_ROOT, "**"), recursive=True)
                  if os.path.isdir(p) and os.path.basename(p).lower() == TARGET_NAME]
    assert candidates, "Couldn't find 'mrleyedataset' in the ZIP."
    EXTRACT_DIR = candidates[0]

print("Dataset root:", EXTRACT_DIR)
print("Subdirs:", os.listdir(EXTRACT_DIR))



Downloading...
From (original): https://drive.google.com/uc?id=1Zr_AtNJsZ1a4QNipHvAGddCc7XMUJUUX
From (redirected): https://drive.google.com/uc?id=1Zr_AtNJsZ1a4QNipHvAGddCc7XMUJUUX&confirm=t&uuid=cd57338c-2ac4-4693-9c1a-d825260f31f4
To: /content/archive.zip
100%|██████████| 346M/346M [00:01<00:00, 267MB/s]


Dataset root: /content/data/mrleyedataset
Subdirs: ['Close-Eyes', 'Open-Eyes']


In [13]:
IMG_SIZE, BATCH, SEED = 160, 32, 1337
VAL_SPLIT = 0.10
EPOCHS_WARMUP, EPOCHS_FINETUNE = 5, 10

from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(EXTRACT_DIR, label_mode="binary",
    validation_split=VAL_SPLIT, subset="training", seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH, shuffle=True)
val_ds = image_dataset_from_directory(EXTRACT_DIR, label_mode="binary",
    validation_split=VAL_SPLIT, subset="validation", seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH, shuffle=True)

AUTOTUNE = tf.data.AUTOTUNE
train_ds, val_ds = train_ds.cache().prefetch(AUTOTUNE), val_ds.cache().prefetch(AUTOTUNE)

from tensorflow.keras import layers
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.2),
])

Found 84898 files belonging to 2 classes.
Using 76409 files for training.
Found 84898 files belonging to 2 classes.
Using 8489 files for validation.


In [14]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import Model

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = augment(inputs)
x = layers.Lambda(preprocess_input)(x)
base = MobileNetV2(include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3))
base.trainable = False
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=[tf.keras.metrics.BinaryAccuracy(name="acc"),
                       tf.keras.metrics.Precision(name="prec"),
                       tf.keras.metrics.Recall(name="rec")])

cb_es = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor="val_acc")
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_WARMUP, callbacks=[cb_es])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 326s 134ms/step - acc: 0.9443 - loss: 0.1443 - prec: 0.9534 - rec: 0.9353 - val_acc: 0.9714 - val_loss: 0.0756 - val_prec: 0.9709 - val_rec: 0.9725
Epoch 2/5
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 318s 133ms/step - acc: 0.9656 - loss: 0.0923 - prec: 0.9696 - rec: 0.9621 - val_acc: 0.9737 - val_loss: 0.0676 - val_prec: 0.9809 - val_rec: 0.9669
Epoch 3/5
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 321s 134ms/step - acc: 0.9691 - loss: 0.0838 - prec: 0.9716 - rec: 0.9671 - val_acc: 0.9741 - val_loss: 0.0647 - val_prec: 0.9818 - val_rec: 0.9667
Epoch 4/5
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 321s 135ms/step - acc: 0.9705 - loss: 0.0780 - prec: 0.9733 - rec: 0.9682 - val_acc: 0.9756 - val_loss: 0.0620 - val_prec: 0.9780 - val_rec: 0.9737
Epoch 5/5
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 322s 135ms/step - acc: 0.9719 - loss: 0.0748 - prec: 0.9734 - rec: 0.9708 - val_acc: 0.9783 - val_loss: 0.0596 - val_prec: 0.9766 - val_rec: 0.9807


In [15]:
# fine-tune
base.trainable = True
for l in base.layers[:-40]:
    l.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(5e-5),
              loss="binary_crossentropy",
              metrics=[tf.keras.metrics.BinaryAccuracy(name="acc"),
                       tf.keras.metrics.Precision(name="prec"),
                       tf.keras.metrics.Recall(name="rec")])
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FINETUNE, callbacks=[cb_es])

print("Eval:")
model.evaluate(val_ds)

Epoch 1/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 451s 185ms/step - acc: 0.9533 - loss: 0.1567 - prec: 0.9479 - rec: 0.9617 - val_acc: 0.9627 - val_loss: 0.1165 - val_prec: 0.9502 - val_rec: 0.9774
Epoch 2/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 444s 186ms/step - acc: 0.9820 - loss: 0.0507 - prec: 0.9803 - rec: 0.9842 - val_acc: 0.9530 - val_loss: 0.1166 - val_prec: 0.9942 - val_rec: 0.9124
Epoch 3/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 442s 185ms/step - acc: 0.9856 - loss: 0.0401 - prec: 0.9831 - rec: 0.9884 - val_acc: 0.9697 - val_loss: 0.0767 - val_prec: 0.9572 - val_rec: 0.9842
Epoch 4/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 441s 185ms/step - acc: 0.9869 - loss: 0.0344 - prec: 0.9847 - rec: 0.9895 - val_acc: 0.9782 - val_loss: 0.0777 - val_prec: 0.9916 - val_rec: 0.9651
Epoch 5/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 445s 186ms/step - acc: 0.9885 - loss: 0.0323 - prec: 0.9866 - rec: 0.9906 - val_acc: 0.9868 - val_loss: 0.0354 - val_prec: 0.9820 - val_rec: 0.9921
Epoch 6/10
2388/2388 ━━━━━━━━━━━━━━━━━━━━ 444s 186

[0.035393550992012024,
 0.9868064522743225,
 0.9820193648338318,
 0.9920819997787476]

In [17]:
# export for VS Code inference
import os, json # Import json here as it's used later in the cell

EXPORT_DIR = "./mobilenetv2_eye_state"
os.makedirs(EXPORT_DIR, exist_ok=True)
model.save(os.path.join(EXPORT_DIR, "model.keras")) # Save in Keras native format
model.export(os.path.join(EXPORT_DIR, "savedmodel")) # Export in SavedModel format

class_names = dict(enumerate(sorted(os.listdir(EXTRACT_DIR))))
with open(os.path.join(EXPORT_DIR, "labels.json"), "w") as f:
    json.dump(class_names, f, indent=2)

Saved artifact at './mobilenetv2_eye_state/savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135770710960912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710961296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710960528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710961680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710964368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710965904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710965136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710966096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278416: TensorSpec(shape=(), dtype=tf.resource, name=None

In [18]:
# Optional TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
open(os.path.join(EXPORT_DIR, "model.tflite"), "wb").write(converter.convert())
print("Saved to", EXPORT_DIR)

Saved artifact at '/tmp/tmpxz44jr00'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135770710964368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710965904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710965136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770710966096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028277840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028278800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135770028279952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13577002828033

In [1]:
# just run:
model.evaluate(val_ds)


NameError: name 'model' is not defined